In [64]:
import json
import time
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
import tensorflow_hub as hub
from datetime import datetime
import re
from tensorflow import keras
import itertools
import math
import shutil
from google.colab import files
import requests
import urllib.request

# Confusion Matrix
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.metrics import confusion_matrix

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
class bcolors:
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    PURPLE = '\033[95m'
    END = '\033[0m'
    WHITE_BACKGROUND = '\033[0;30;47m'

In [5]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 4.1MB/s 


In [0]:
# BERT
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#@title Choose the Hyper-Parameter To Run
#@markdown From The Hyper Parameter Fine Tuning, The Best Configuartion Has The Following Parameter
K=5 #@param {type:"number"}
BEST_EPOCHS = 1.0 #@param {type:"number"}
BEST_LEARNING_RATE = 2e-05 #@param {type:"number"}
BEST_BATCH_SIZE = 32 #@param {type:"number"}
BEST_WARMUP_PROPORTION = 0.1 #@param {type:"number"}
BEST_MAX_SEQ_LENGTH = 128 #@param {type:"number"}
OUTPUT_DIR = '/content/drive/My Drive/BERT/'#@param {type:"string"}

In [0]:
#BERT version
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1"

In [0]:
#BERT
BERT_ACCURACY = []
BERT_PRECISION = []
BERT_F1_SCORE = []
BERT_RECALL = []
BERT_AUC = []
BERT_LOSS = []
BERT_TP = []
BERT_TN = []
BERT_FP = []
BERT_FN = []

#VALIDATOR
PREDICTION_RIGHT = []
PREDICTION_TOTAL = []
PREDICTION_THRESHOLD_1 = []
PREDICTION_THRESHOLD_3 = []
PREDICTION_THRESHOLD_6 = []
VAL_ACCURACY = []
VAL_PRECISION = []
VAL_F1_SCORE = []
VAL_RECALL = []
VAL_TP = []
VAL_TN = []
VAL_FP = []
VAL_FN = []

#time
TIME = []

#dataframes
complete_df = pd.DataFrame(columns=['data_quantity', 'iteration', 
                                    'bert_accuracy', 'bert_precision', 'bert_f1_score', 'bert_recall', 
                                    'bert_auc', 'bert_loss', 'bert_tp', 'bert_tn', 'bert_fp', 'bert_fn', 
                                    'prediction_right', 'prediction_total', 
                                    'prediction_threshold_1', 'prediction_threshold_3', 'prediction_threshold_6', 
                                    'val_accuracy', 'val_precision', 'val_f1_score', 'val_recall', 'val_tp', 'val_tn', 'val_fp', 'val_fn',
                                    'time'])

In [71]:
complete_df.head()

,data_quantity,iteration,bert_accuracy,bert_precision,bert_f1_score,bert_recall,bert_auc,bert_loss,bert_tp,bert_tn,bert_fp,bert_fn,prediction_right,prediction_total,prediction_threshold_1,prediction_threshold_3,prediction_threshold_6,val_accuracy,val_precision,val_f1_score,val_recall,val_tp,val_tn,val_fp,val_fn,time


In [0]:
def Average(lst):
  return sum(lst) / len(lst) 

In [0]:
MY_DRIVE = '/content/drive/My Drive'

In [0]:
df_restored = pd.read_pickle(MY_DRIVE+"/BERT/data/wiki-italian-78K-train-test.pickle")
real_test_unbalanced_restored = pd.read_pickle(MY_DRIVE+"/BERT/data/italian_wiki_real_test-3631.pickle")

In [83]:
real_test_unbalanced_restored.shape

(3631, 5)

In [84]:
df_restored.shape

(78216, 5)

In [85]:
df_restored.head()

,first_sentence_id,second_sentence_id,first_sentence,second_sentence,two_sentences_in_same_segment
0,6037,6038,"Tra i corridori c'erano il futuro vincitore, M...",La corsa catturò l'attenzione del pubblico. Le...,True
1,3067,3068,Nel 1908 si iscrisse all'università di Oslo do...,Al suo ritorno la sua reputazione scientifica ...,True
2,20299,15140,Il Logicismo si avviò ad essere superato quand...,"Il territorio, intensamente coltivato, present...",False
3,10485,10490,La maggior parte dei buffer contengono testo c...,L'utente invoca comandi per la modifica del te...,True
4,754,755,"Tiberio, invece, si impegnò a mantenersi infor...","Il prefetto del pretorio, infatti, iniziò a pe...",True


In [86]:
df_restored.two_sentences_in_same_segment.value_counts()

True     39108
False    39108
Name: two_sentences_in_same_segment, dtype: int64

In [87]:
real_test_unbalanced_restored.two_sentences_in_same_segment.value_counts()

True     2390
False    1241
Name: two_sentences_in_same_segment, dtype: int64

In [0]:
val_set = real_test_unbalanced_restored

If sub-sampling needed to reduce the computation time, run the following cells

In [0]:
#df_sub_sampled = df_restored.sample(4000)
#df_sub_sampled.shape

In [0]:
#df_sub_sampled.two_sentences_in_same_segment.value_counts()

Train-test-val split 

In [0]:
dev_train_test_set = df_restored
#dev_train_test_set = df_restored.sample(int(len(df_restored) *0.8))
#val_set = df_restored[~df_restored.index.isin(dev_train_test_set.index)]

In [91]:
dev_train_test_set.shape

(78216, 5)

In [92]:
val_set.shape

(3631, 5)

In [93]:
dev_train_test_set.two_sentences_in_same_segment.value_counts()

True     39108
False    39108
Name: two_sentences_in_same_segment, dtype: int64

In [94]:
dev_train_test_set.head()

,first_sentence_id,second_sentence_id,first_sentence,second_sentence,two_sentences_in_same_segment
0,6037,6038,"Tra i corridori c'erano il futuro vincitore, M...",La corsa catturò l'attenzione del pubblico. Le...,True
1,3067,3068,Nel 1908 si iscrisse all'università di Oslo do...,Al suo ritorno la sua reputazione scientifica ...,True
2,20299,15140,Il Logicismo si avviò ad essere superato quand...,"Il territorio, intensamente coltivato, present...",False
3,10485,10490,La maggior parte dei buffer contengono testo c...,L'utente invoca comandi per la modifica del te...,True
4,754,755,"Tiberio, invece, si impegnò a mantenersi infor...","Il prefetto del pretorio, infatti, iniziò a pe...",True


In [96]:
val_set.head()

,first_sentence_id,second_sentence_id,first_sentence,second_sentence,two_sentences_in_same_segment
0,3320.txt_1,3320.txt_3,È passato alla storia per aver soppresso l'ord...,"Nativo del Bazadais, in Guascogna, era figlio ...",False
1,3320.txt_3,3320.txt_4,"Nativo del Bazadais, in Guascogna, era figlio ...",A Lione Bertrand fu vicario generale del frate...,True
2,3320.txt_4,3320.txt_6,A Lione Bertrand fu vicario generale del frate...,Da arcivescovo venne eletto papa il 5 giugno 1...,False
3,3320.txt_6,3320.txt_8,Da arcivescovo venne eletto papa il 5 giugno 1...,"Invece di ritornare a Roma, che allora era dil...",False
4,3320.txt_8,3320.txt_9,"Invece di ritornare a Roma, che allora era dil...","Bertrand non era né italiano né cardinale, e l...",True


In [97]:
print('Starting Experiment')
DATA_QUANTITY = len(dev_train_test_set)
print(f'Beginning dev train plus dev test for the {DATA_QUANTITY}-dataset')
start_iteration = datetime.now()
print("Start time: " + str(start_iteration))

Starting Experiment
Beginning dev train plus dev test for the 78216-dataset
Start time: 2019-11-12 14:27:12.556581


In [98]:
test = dev_train_test_set.sample(int(DATA_QUANTITY / 2))
train = dev_train_test_set[~dev_train_test_set.index.isin(test.index)]
print("train shape: " + str(train.shape))
print("test shape: " + str(test.shape))

train shape: (39108, 5)
test shape: (39108, 5)


In [100]:
train.two_sentences_in_same_segment.value_counts()

True     19593
False    19515
Name: two_sentences_in_same_segment, dtype: int64

In [101]:
test.two_sentences_in_same_segment.value_counts()

False    19593
True     19515
Name: two_sentences_in_same_segment, dtype: int64

In [102]:
train.columns

Index(['first_sentence_id', 'second_sentence_id', 'first_sentence',
       'second_sentence', 'two_sentences_in_same_segment'],
      dtype='object')

In [0]:
#Define the column to be used inside BERT
DATA_COLUMN_1 = 'first_sentence'
LABEL_COLUMN = 'two_sentences_in_same_segment'
DATA_COLUMN_2 = 'second_sentence'
ID_COLUMN_1 = 'first_sentence_id'
ID_COLUMN_2 = 'second_sentence_id'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [True, False]

In [0]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid = None, # Globally unique ID for bookkeeping, unused in this example
                                                                         text_a = x[DATA_COLUMN_1], 
                                                                         text_b = x[DATA_COLUMN_2], 
                                                                         label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid = None, 
                                                                         text_a = x[DATA_COLUMN_1], 
                                                                         text_b = x[DATA_COLUMN_2], 
                                                                         label = x[LABEL_COLUMN]), axis = 1)



In [105]:
train_InputExamples.head()

1     <bert.run_classifier.InputExample object at 0x...
4     <bert.run_classifier.InputExample object at 0x...
10    <bert.run_classifier.InputExample object at 0x...
11    <bert.run_classifier.InputExample object at 0x...
14    <bert.run_classifier.InputExample object at 0x...
dtype: object

In [0]:
########## BERT TOKENIZER AND MODEL ##########

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])

  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)



In [107]:
tokenizer = create_tokenizer_from_hub_module()
# We'll set sequences to be at most 128 tokens long.

# HPFN
MAX_SEQ_LENGTH = BEST_MAX_SEQ_LENGTH

# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 39108


INFO:tensorflow:Writing example 0 of 39108


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Writing example 10000 of 39108


INFO:tensorflow:Writing example 20000 of 39108


INFO:tensorflow:Writing example 20000 of 39108


INFO:tensorflow:Writing example 30000 of 39108


INFO:tensorflow:Writing example 30000 of 39108


In [0]:
# Model Creation
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)
    # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    
    #EMRE MODIFIED THIS
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    #log_probs = tf.nn.softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    
    ## additional casting is needed as done below, otherwise gets "Tensor conversion requested dtype int32 for Tensor with dtype int64 - while estimator.export_savedmodel" error
    input_ids = tf.cast(input_ids, tf.int32)
    input_mask = tf.cast(input_mask, tf.int32)
    segment_ids = tf.cast(segment_ids, tf.int32)
    label_ids = tf.cast(label_ids, tf.int32)

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [110]:
########## BERT HYPER-PARAMETERS ##########

# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)

# HPFN
BATCH_SIZE = BEST_BATCH_SIZE
LEARNING_RATE = BEST_LEARNING_RATE
NUM_TRAIN_EPOCHS = BEST_EPOCHS
# Warmup is a period of time where the learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = BEST_WARMUP_PROPORTION


# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
print("num_train_steps:"+str(num_train_steps))


num_train_steps:1222


In [111]:
OUTPUT_DIR

'/content/drive/My Drive/BERT/'

In [112]:
# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)



INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/BERT/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3f76858cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/BERT/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3f76858cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [113]:
num_train_steps

1222

In [114]:
 ########## TRAINING ##########

# Training
current_time = datetime.now()
print(f'{bcolors.BOLD}{bcolors.PURPLE}Beginning Training! ({current_time}){bcolors.END}')
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print(f"\nTraining ended in {datetime.now() - current_time}\n")


Beginning Training! (2019-11-12 14:33:53.197557)
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 1222 into /content/drive/My Drive/BERT/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1222 into /content/drive/My Drive/BERT/model.ckpt.


INFO:tensorflow:Loss for final step: 0.06306076.


INFO:tensorflow:Loss for final step: 0.06306076.



Training ended in 0:08:47.172617



In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [116]:
########## ESTIMATION ##########

# Estimation
est = estimator.evaluate(input_fn=test_input_fn, steps=None)
print(f'BERT Accuracy: {est["eval_accuracy"]}')

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 1222: auc = 0.97763026, eval_accuracy = 0.9776261, f1_score = 0.9776231, false_negatives = 479.0, false_positives = 396.0, global_step = 1222, loss = 0.070539705, precision = 0.9797027, recall = 0.9755525, true_negatives = 19119.0, true_positives = 19114.0


INFO:tensorflow:Saving dict for global step 1222: auc = 0.97763026, eval_accuracy = 0.9776261, f1_score = 0.9776231, false_negatives = 479.0, false_positives = 396.0, global_step = 1222, loss = 0.070539705, precision = 0.9797027, recall = 0.9755525, true_negatives = 19119.0, true_positives = 19114.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1222: /content/drive/My Drive/BERT/model.ckpt-1222


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1222: /content/drive/My Drive/BERT/model.ckpt-1222


BERT Accuracy: 0.9776260852813721


In [0]:
########## PREDICTION FUNCTION ##########

# Prediction Function
import traceback
def getPrediction(val_set):
  col_first_second_sentences = val_set[['first_sentence','second_sentence']].values.tolist()
  col_first_sentences = val_set['first_sentence'].values.tolist()
  col_second_sentences = val_set['second_sentence'].values.tolist()
  col_first_sentence_ids = val_set['first_sentence_id'].values.tolist()
  col_second_sentence_ids = val_set['second_sentence_id'].values.tolist()
  col_ground_truths =  val_set['two_sentences_in_same_segment'].values.tolist()
  label_list = [True, False]
  input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = x[1], label = False) for x in col_first_second_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  #predictions = estimator.predict(predict_input_fn,predict_keys="probabilities")
  predictions = estimator.predict(predict_input_fn)
  #pred_probs = [prediction['probabilities'] for prediction in predictions]
  #pred_labels = [label_list[prediction['labels']] for prediction in predictions]
  zipped = list(zip(col_first_sentence_ids,col_second_sentence_ids,col_first_sentences,col_second_sentences,col_ground_truths,predictions))
  return zipped
  #return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(col_first_second_sentencein_sentences, predictions)]
  #return [(sentence, prediction) for sentence, prediction in zip(in_sentences, predictions)]
#est['eval_accuracy']

In [158]:
val_set.shape

(3631, 5)

In [119]:
val_set.head()

,first_sentence_id,second_sentence_id,first_sentence,second_sentence,two_sentences_in_same_segment
0,3320.txt_1,3320.txt_3,È passato alla storia per aver soppresso l'ord...,"Nativo del Bazadais, in Guascogna, era figlio ...",False
1,3320.txt_3,3320.txt_4,"Nativo del Bazadais, in Guascogna, era figlio ...",A Lione Bertrand fu vicario generale del frate...,True
2,3320.txt_4,3320.txt_6,A Lione Bertrand fu vicario generale del frate...,Da arcivescovo venne eletto papa il 5 giugno 1...,False
3,3320.txt_6,3320.txt_8,Da arcivescovo venne eletto papa il 5 giugno 1...,"Invece di ritornare a Roma, che allora era dil...",False
4,3320.txt_8,3320.txt_9,"Invece di ritornare a Roma, che allora era dil...","Bertrand non era né italiano né cardinale, e l...",True


In [120]:
val_cols = val_set[['first_sentence','second_sentence']].values.tolist()
val_cols[0:2]

[["È passato alla storia per aver soppresso l'ordine dei Templari (1307) e per aver spostato la Santa Sede in Francia: Clemente V infatti dimorò, sin dal momento dell'elezione, in Francia (Lione, Vienne, Poitiers) e infine trasferì la Curia papale nella città di Carpentras nel 1313, quasi un anno prima di morire, poiché Carpentras si trovava all'interno del Contado Venassino, feudo papale non soggetto all'autorità di Filippo il Bello. Dopo la sua morte (1314) e il lungo Conclave di Lione, il suo successore Giovanni XXII, nel 1316, preferì trasferire la sede papale e la Curia nella vicina Avignone, dove già risiedeva la Corte pontificia. Fu il primo Papa ad assumere la tiara.",
  'Nativo del Bazadais, in Guascogna, era figlio di Béraud de Got e di Ida di Blanquefort. Era anche fratello di Bérard de Got, arcivescovo di Lione, e zio di Raymond de Got, cardinale diacono di Santa Maria Nuova.'],
 ['Nativo del Bazadais, in Guascogna, era figlio di Béraud de Got e di Ida di Blanquefort. Era a

In [174]:
val_set.iloc[0]

first_sentence_id                                                       3320.txt_1
second_sentence_id                                                      3320.txt_3
first_sentence                   È passato alla storia per aver soppresso l'ord...
second_sentence                  Nativo del Bazadais, in Guascogna, era figlio ...
two_sentences_in_same_segment                                                False
Name: 0, dtype: object

In [183]:
predictions = []
zipped = getPrediction(val_set)

INFO:tensorflow:Writing example 0 of 3631


INFO:tensorflow:Writing example 0 of 3631


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/BERT/model.ckpt-1222


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [184]:
zipped

[('3320.txt_1',
  '3320.txt_3',
  "È passato alla storia per aver soppresso l'ordine dei Templari (1307) e per aver spostato la Santa Sede in Francia: Clemente V infatti dimorò, sin dal momento dell'elezione, in Francia (Lione, Vienne, Poitiers) e infine trasferì la Curia papale nella città di Carpentras nel 1313, quasi un anno prima di morire, poiché Carpentras si trovava all'interno del Contado Venassino, feudo papale non soggetto all'autorità di Filippo il Bello. Dopo la sua morte (1314) e il lungo Conclave di Lione, il suo successore Giovanni XXII, nel 1316, preferì trasferire la sede papale e la Curia nella vicina Avignone, dove già risiedeva la Corte pontificia. Fu il primo Papa ad assumere la tiara.",
  'Nativo del Bazadais, in Guascogna, era figlio di Béraud de Got e di Ida di Blanquefort. Era anche fratello di Bérard de Got, arcivescovo di Lione, e zio di Raymond de Got, cardinale diacono di Santa Maria Nuova.',
  False,
  {'labels': 0,
   'probabilities': array([-0.6204713 , 

In [0]:
df_pred = pd.DataFrame(zipped, columns = ["first_sentence_id","second_sentence_id","first_sentence","second_sentence","ground_truth","prediction"]) 

In [189]:
label_list

[True, False]

In [197]:
df_pred["pred_label"] = df_pred["prediction"].apply(lambda x: label_list[x['labels']])
df_pred["pred_prob"] = df_pred["prediction"].apply(lambda x: x['probabilities'])
df_pred.head()

,first_sentence_id,second_sentence_id,first_sentence,second_sentence,ground_truth,prediction,pred_label,pred_prob
0,3320.txt_1,3320.txt_3,È passato alla storia per aver soppresso l'ord...,"Nativo del Bazadais, in Guascogna, era figlio ...",False,"{'probabilities': [-0.6204713, -0.77152157], '...",True,"[-0.6204713, -0.77152157]"
1,3320.txt_3,3320.txt_4,"Nativo del Bazadais, in Guascogna, era figlio ...",A Lione Bertrand fu vicario generale del frate...,True,"{'probabilities': [-0.0019317077, -6.250343], ...",True,"[-0.0019317077, -6.250343]"
2,3320.txt_4,3320.txt_6,A Lione Bertrand fu vicario generale del frate...,Da arcivescovo venne eletto papa il 5 giugno 1...,False,"{'probabilities': [-0.0016323111, -6.418591], ...",True,"[-0.0016323111, -6.418591]"
3,3320.txt_6,3320.txt_8,Da arcivescovo venne eletto papa il 5 giugno 1...,"Invece di ritornare a Roma, che allora era dil...",False,"{'probabilities': [-0.0024591226, -6.009203], ...",True,"[-0.0024591226, -6.009203]"
4,3320.txt_8,3320.txt_9,"Invece di ritornare a Roma, che allora era dil...","Bertrand non era né italiano né cardinale, e l...",True,"{'probabilities': [-0.015433289, -4.178935], '...",True,"[-0.015433289, -4.178935]"


In [203]:
df_pred[0:30]

,first_sentence_id,second_sentence_id,first_sentence,second_sentence,ground_truth,prediction,pred_label,pred_prob
0,3320.txt_1,3320.txt_3,È passato alla storia per aver soppresso l'ord...,"Nativo del Bazadais, in Guascogna, era figlio ...",False,"{'probabilities': [-0.6204713, -0.77152157], '...",True,"[-0.6204713, -0.77152157]"
1,3320.txt_3,3320.txt_4,"Nativo del Bazadais, in Guascogna, era figlio ...",A Lione Bertrand fu vicario generale del frate...,True,"{'probabilities': [-0.0019317077, -6.250343], ...",True,"[-0.0019317077, -6.250343]"
2,3320.txt_4,3320.txt_6,A Lione Bertrand fu vicario generale del frate...,Da arcivescovo venne eletto papa il 5 giugno 1...,False,"{'probabilities': [-0.0016323111, -6.418591], ...",True,"[-0.0016323111, -6.418591]"
3,3320.txt_6,3320.txt_8,Da arcivescovo venne eletto papa il 5 giugno 1...,"Invece di ritornare a Roma, che allora era dil...",False,"{'probabilities': [-0.0024591226, -6.009203], ...",True,"[-0.0024591226, -6.009203]"
4,3320.txt_8,3320.txt_9,"Invece di ritornare a Roma, che allora era dil...","Bertrand non era né italiano né cardinale, e l...",True,"{'probabilities': [-0.015433289, -4.178935], '...",True,"[-0.015433289, -4.178935]"
5,3320.txt_9,3320.txt_11,"Bertrand non era né italiano né cardinale, e l...","All'inizio del 1306, Clemente abrogò di fatto ...",False,"{'probabilities': [-0.016432194, -4.1167197], ...",True,"[-0.016432194, -4.1167197]"
6,3320.txt_11,3320.txt_12,"All'inizio del 1306, Clemente abrogò di fatto ...",Il 13 ottobre 1307 fu ordinato l'arresto di tu...,True,"{'probabilities': [-0.0017939203, -6.324233], ...",True,"[-0.0017939203, -6.324233]"
7,3320.txt_12,3320.txt_13,Il 13 ottobre 1307 fu ordinato l'arresto di tu...,"L'8 settembre 1308, con la Bolla papale ""Super...",True,"{'probabilities': [-0.0014456784, -6.539864], ...",True,"[-0.0014456784, -6.539864]"
8,3320.txt_13,3320.txt_14,"L'8 settembre 1308, con la Bolla papale ""Super...",Approfittando della sua influenza sul pontefic...,True,"{'probabilities': [-0.08809341, -2.4730809], '...",True,"[-0.08809341, -2.4730809]"
9,3320.txt_14,3320.txt_15,Approfittando della sua influenza sul pontefic...,"Nel perseguimento dei desideri del re, Clement...",True,"{'probabilities': [-0.0013650871, -6.5971913],...",True,"[-0.0013650871, -6.5971913]"


In [199]:
df_pred.pred_label.value_counts()

True     3220
False     411
Name: pred_label, dtype: int64

In [204]:
df_pred.ground_truth.value_counts()

True     2390
False    1241
Name: ground_truth, dtype: int64

In [0]:
def get_precision_recall_f1(df_set,ground_truth,pred_label):
  ########## TP - TN - FP - FN ##########

  # Dataframe for TP, TN, FP, FN
  tp_df = df_set.query('ground_truth == True and pred_label == True')
  tp = len(tp_df)
  tn_df = df_set.query('ground_truth == False and pred_label == False')
  tn = len(tn_df)
  fp_df = df_set.query('ground_truth == False and pred_label == True')
  fp = len(fp_df)
  fn_df = df_set.query('ground_truth == True and pred_label == False')
  fn = len(fn_df)

  ########## VALIDATOR ESTIMATION ##########

  # Validator Estimation

  # Accuracy, Precision, Recall, F1-Score
  accuracy = (tp + tn) / (tp + tn + fp + fn)
  if tp != 0 or fp != 0:
    precision = tp / (tp + fp)
  else:
    precision = 0

  if tp != 0 or fn != 0:
    recall = tp / (tp + fn)
  else:
    recall = 0

  if precision != 0 or recall != 0:
    f1_score = 2 * precision * recall / (precision + recall)
  else:
    f1_score = 0
  print(f'Validator\nAccuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1-Score: {f1_score}')

#####

In [202]:
get_precision_recall_f1(df_pred,"ground_truth","pred_label")

Validator
Accuracy: 0.7267970256127788
Precision: 0.7170807453416149
Recall: 0.9661087866108786
F1-Score: 0.8231729055258467


In [0]:
def remove_escape_char(sentence):
  sentence = sentence.replace("\\","")
  sentence = sentence.replace("\"","")
  return sentence

In [0]:
print(remove_escape_char("What\'s Following Me?"))

What's Following Me?


## **Generate segment files in txt format**

In [0]:
output_lines = []

f = open("/content/drive/My Drive/BERT/data/pred_res_2.txt","w+")
cnt = 0
for (sentence_couple, prob, label) in predictions_unlabeled:
  cnt += 1
  first_sentence = remove_escape_char(sentence_couple[0])
  second_sentence = remove_escape_char(sentence_couple[1])
  
  if not label:
    f.write(first_sentence+"\n")
    f.write("=======\n")
    output_lines.append(first_sentence)
    output_lines.append("=======")
  else:
    f.write(first_sentence+"\n")
    output_lines.append(first_sentence)
  if(cnt==len(predictions_unlabeled)):
    f.write(second_sentence+"\n")
    output_lines.append(second_sentence)
f.close()



In [0]:
output_lines

['Eleanor McEvoy',
 "Eleanor McEvoy (born 22 January 1967) is one of Ireland's most accomplished contemporary singer/songwriters.",
 "McEvoy composed the song Only A Woman's Heart, title track of A Woman's Heart, the best-selling Irish album in Irish history.",
 "McEvoy's life as a musician began at the age of four when she began playing piano.",
 'At the age of eight she took up violin.',
 'Upon finishing school she attended Trinity College, Dublin where she studied music by day and worked in pit orchestras and music clubs by night.',
 'McEvoy graduated from Trinity with an Honors Degree in music, and spent four months busking in New York City.',
 'In 1988, she was accepted into the RTÉ National Symphony Orchestra where she spent four years before leaving to concentrate on songwriting.',
 'She built up a following in clubs in Dublin with her three piece band, Jim Tate on bass, Noel Eccles on drums, and latterly Bill Shanley on guitar.',
 "During a solo date in July 1992, she performed

In [0]:
predictions_unlabeled[0]

(['Eleanor McEvoy',
  "Eleanor McEvoy (born 22 January 1967) is one of Ireland's most accomplished contemporary singer/songwriters."],
 array([-0.02966756, -3.5324967 ], dtype=float32),
 True)

In [0]:
# run this cell if pred prob enabled (predictions = estimator.predict(predict_input_fn,predict_keys="probabilities"))
# for (sentence_couple, probs) in predictions:
#  print(probs['probabilities'][0])

In [0]:
val_set[0:2]

In [0]:
def serving_input_receiver_fn():
  feature_spec = {
      "input_ids" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
      "input_mask" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
      "segment_ids" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
      "label_ids" :  tf.FixedLenFeature([], tf.int64)
  }
  serialized_tf_example = tf.placeholder(dtype=tf.string,
                                         shape=[None],
                                         name='input_example_tensor')
  print(serialized_tf_example.shape)
  receiver_tensors = {'example': serialized_tf_example}
  features = tf.parse_example(serialized_tf_example, feature_spec)
  return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

estimator._export_to_tpu = False  # this is important
estimator.export_saved_model(export_dir_base='/content/drive/My Drive/BERT',serving_input_receiver_fn=serving_input_receiver_fn)

In [0]:
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model import signature_constants
export_path = estimator.export_saved_model(
  "/content/drive/My Drive/BERT/", serving_input_receiver_fn)
with tf.Session() as sess:
  imported = tf.saved_model.load(export_dir=export_path, sess=sess, tags=[tag_constants.SERVING])


In [0]:
try:
  estimator._export_to_tpu = False
  estimator.export_saved_model('/content/drive/My Drive/BERT/', serving_input_receiver_fn)
except Exception as ex:
  print(str(ex))

In [0]:
def serving_input_fn():
  try:
    label_ids = tf.placeholder(tf.int32, [None], name='label_ids')
    input_ids = tf.placeholder(tf.int32, [None, FLAGS.max_seq_length], name='input_ids')
    input_mask = tf.placeholder(tf.int32, [None, FLAGS.max_seq_length], name='input_mask')
    segment_ids = tf.placeholder(tf.int32, [None, FLAGS.max_seq_length], name='segment_ids')
    input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn({
        'label_ids': label_ids,
        'input_ids': input_ids,
        'input_mask': input_mask,
        'segment_ids': segment_ids,
    })()
  except Exception as ex:
    print(str(ex))
  return input_fn

In [0]:
from tensorflow.contrib import predictor
predict_fn = predictor.from_saved_model('/content/drive/My Drive/BERT/')
